<img style="float: center;" src="../images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Brian Kim, Avishek Kumar, and Jonathan Morgan.

# Data Preparation for Machine Learning - Feature Creation
----

## Python Setup
- Back to [Table of Contents](#Table-of-Contents)

Before we begin, run the code cell below to initialize the libraries we'll be using in this assignment. We're already familiar with `numpy`, `pandas`, and `psycopg2` from previous tutorials. Here we'll also be using [`scikit-learn`](http://scikit-learn.org) to fit modeling.

In [ ]:
%pylab inline
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"

In [ ]:
# Insert team schema name below:
myschema = 'ada_18_uchi'

## Creating Features

Our features are our independent variables or predictors. Good features make machine learning systems effective. 
The better the features the easier it is the capture the structure of the data. You generate features using domain knowledge. In general, it is better to have more complex features and a simpler model rather than vice versa. Keeping the model simple makes it faster to train and easier to understand rather then extensively searching for the "right" model and "right" set of parameters. 

Machine Learning Algorithms learn a solution to a problem from sample data. The set of features is the best representation of the sample data to learn a solution to a problem. 

- **Feature engineering** is the process of transforming raw data into features that better represent the underlying problem/data/structure  to the predictive models, resulting in improved model accuracy on unseen data." ( from [Discover Feature Engineering](http://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/) ).  In text, for example, this might involve deriving traits of the text like word counts, verb counts, or topics to feed into a model rather than simply giving it the raw text.

Example of feature engineering are: 

- **Transformations**, such a log, square, and square root.
- **Dummy (binary) variables**, also known as *indicator variables*, often done by taking categorical variables
(such as city) which do not have a numerical value, and adding them to models as a binary value.
- **Discretization**. Several methods require features to be discrete instead of continuous. This is often done 
by binning, which you can do by equal width. 
- **Aggregation.** Aggregate features often constitute the majority of features for a given problem. These use 
different aggregation functions (*count, min, max, average, standard deviation, etc.*) which summarize several
values into one feature, aggregating over varying windows of time and space. For example, given urban data, 
we would want to calculate the *number* (and *min, max, mean, variance*, etc.) of crimes within an *m*-mile radius
of an address in the past *t* months for varying values of *m* and *t*, and then use all of them as features.

>Our preliminary features are the following
>- `new_employer` (Binary): 1 if the employer is "new" (did not exist before a cutoff date), 0 if they are not. 
>- `nb_emp` (Aggregation): Total number of employees working for a given employer at a given year and quarter.
>- `total_payroll` (Aggregation): Total amount paid out by the employer in earnings during a given year adn quarter. 
>- `wage_gap` (Aggregation): TO DO

## New vs Old Employers

Let's create a first binary feature defining "old" and "new" firms. Old firms are determined according to age cutoff, with a default value is 5 years.

### Step by Step Approach

In [ ]:
conn = psycopg2.connect(database=db_name, host = hostname)
cursor = conn.cursor()

Let's start by creating a list of IDs that existed 5 years prior to Q1 of 2013.

In [ ]:
sql = '''
CREATE TEMP TABLE features_prior_ids AS
SELECT CONCAT(ein, '-', seinunit, '-', empr_no) AS id
FROM il_des_kcmo.il_qcew_employers
WHERE year = 2013-5 AND quarter = 1;
COMMIT;
'''
cursor.execute(sql)

We then merge this list to our labels: employers from the labels table that do not match any of the prior ID's will be flagged as "new" (they did not exist 5 years ago), while the other ones will be flagged as old.

In [ ]:
sql = '''
SELECT a.id, CASE WHEN b.id IS NULL THEN 1 ELSE 0 END AS is_new
FROM ada_18_uchi.labels_2013q1_2014q1 AS a
LEFT JOIN features_prior_ids AS b
ON a.id = b.id;
'''
df = pd.read_sql(sql, conn)

Now let's visualize the first rows of data and check how many employers are "new".

In [ ]:
df.head()

In [ ]:
df['is_new'].value_counts()

### Define Function

In order to facilitate creating this feature for several years of data, we combined all the above steps into a Python function, and added a final step that writes the feature table to the database.

In the step-by-step approach above, all SQL queries were entirely hard coded. The Python function however uses parameters so you can easily reuse it. The function's parameters are:
- `year`: The year at which we are doing the prediction.
- `qtr`: The quarter at which we are doing the prediction.
- `delta_t`: The forward-looking window we chose when creating the model's labels. The default value is 1, which means we are prediction at a given time whether an employer will still exist one year later.
- `age_cutoff`: The backwards-looking window we use to define "new" employers. The default value is set to 5, which means "new" employers are defined as existing since less than 5 years.
- `schema`: Your team schema, where the label table will be written. The default value is set to `myschema`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `db_name`: Database name. This is the name of the SQL database we are using. The default value is set to `db_name`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `hostname`: Host name. This is the host name for the SQL database we are using. The default value is set to `hostname`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `overwrite`: Whether you want the function to overwrite tables that already exist. Before writing a table, the function will check whether this table exists, and by default will not overwrite existing tables.

In [ ]:
def employer_age_features(year, qtr, delta_t=1, age_cutoff=5, schema=myschema, 
                          db_name=db_name, hostname=hostname, overwrite=False):
    
    conn = psycopg2.connect(database=db_name, host = hostname) #database connection
    cursor = conn.cursor()
        
    # Let's check if the table already exists:
    cursor.execute('''
    SELECT * FROM information_schema.tables 
    WHERE table_name = 'features_isnew_{year}q{qtr}_age{age_cutoff}' 
    AND table_schema = '{schema}';
    '''.format(year=year, qtr=qtr, age_cutoff=age_cutoff, schema=schema))
    
    # Let's write table if it does not exist (or if overwrite = True)
    if not(cursor.rowcount) or overwrite:
        print("Creating table")    
        
        sql = '''
        CREATE TEMP TABLE features_isnew_{year}q{qtr} AS
        SELECT CONCAT(ein, '-', seinunit, '-', empr_no) AS id
        FROM il_des_kcmo.il_qcew_employers
        WHERE year = {year}-{age_cutoff} AND quarter = {qtr};
        COMMIT;

        DROP TABLE IF EXISTS {schema}.features_isnew_{year}q{qtr}_age{age_cutoff};
        CREATE TABLE {schema}.features_isnew_{year}q{qtr}_age{age_cutoff} AS
        SELECT a.id, CASE WHEN b.id IS NULL THEN 1 ELSE 0 END AS is_new
        FROM {schema}.labels_{year}q{qtr}_{year_pdelta}q{qtr} AS a
        LEFT JOIN features_isnew_{year}q{qtr} AS b
        ON a.id = b.id;
        COMMIT;

        ALTER TABLE {schema}.features_isnew_{year}q{qtr}_age{age_cutoff} OWNER TO {schema}_admin;
        COMMIT;
        '''.format(year=year, qtr=qtr, schema=schema, year_pdelta=year+delta_t, age_cutoff=age_cutoff)  
        
        cursor.execute(sql)
        
    else:
        print("Table already exists")
    
    cursor.close()
    
    sql = '''
    SELECT * FROM {schema}.features_isnew_{year}q{qtr}_age{age_cutoff}
    '''.format(year=year, qtr=qtr, age_cutoff=age_cutoff, schema=schema)
    df = pd.read_sql(sql, conn)  
    
    return df

In [ ]:
df_isnew_2013q1 = employer_age_features(2013, 1)

In [ ]:
df_isnew_2013q1['is_new'].value_counts()

In [ ]:
df_isnew_2014q1 = employer_age_features(2014, 1)

In [ ]:
df_isnew_2014q1['is_new'].value_counts()

## Wages and Employees

Let's create two aggregate features for every employer's total number of employees and total payroll. The features will be created using the QCEW Employer table.

### Step by Step Approach

In [ ]:
conn = psycopg2.connect(database=db_name, host = hostname)
cursor = conn.cursor()

Let's start by querying from the QCEW Employer table the total number of employees and total payroll at a given quarter (Q1 of 2013). 

Since the number of employees is given on a month-by-month basis, let's sum the number for the three months. While this number will not reflect the number of unique employees hired by that employer over the quarter, it will reflect the number of monthly positions' wages paid out by the employer (with individuals who are employed in all three months of that quarter counted 3 times). 

In [ ]:
sql = '''
CREATE TEMP TABLE features_emprs AS
SELECT CONCAT(ein, '-', seinunit, '-', empr_no) AS id, 
        empl_month1::int+empl_month2::int+empl_month3::int AS total_empl,
        total_wages
FROM il_des_kcmo.il_qcew_employers
WHERE year = 2013 AND quarter = 1;
COMMIT;
'''
cursor.execute(sql)

Now let's merge this to our list of labels and visualize the first rows of data.

In [ ]:
sql = '''
SELECT a.id, b.total_empl, b.total_wages
FROM ada_18_uchi.labels_2013q1_2014q1 AS a
LEFT JOIN features_emprs AS b
ON a.id = b.id;
'''
df = pd.read_sql(sql, conn)

In [ ]:
df.head()

Let's also create an "average monthly earnings" feature which will be defined as total payroll divided by total employees, and check summary statistics on the 3 features.

In [ ]:
df['avg_wage'] = df['total_wages']/df['total_empl']

In [ ]:
df[['total_empl', 'total_wages', 'avg_wage']].describe(percentiles=[0.01,0.05,0.25,0.50,0.75,0.95,0.99])

Because of some data inconsistencies in total employees and total wages, some average wages could not be calculated (when `total_empl == 0` and `total_wages == 0`) and some have `inf` values (when `total_empl == 0`). These `NULL` and `inf` values will be problematic for the machine learning algorithm.

Instead of dropping these observations, let's impute the missing values of employees and earnings.

### Imputing Missing Values

There are many ways of imputing missing values based on the rest of the data. Missing values can be imputed to median of the rest of the data, or you can use other characteristics (industry, geography, etc.) to impute an employer's missing value with the median of similar employers.

In this preliminary case, let's impute the missing values to the median value of all the data. Let's start with the Total Number of Employees.

In [ ]:
vals_to_replace = df[df['total_empl']==0]['total_empl'].values
df['total_empl'].replace(vals_to_replace, np.NaN, inplace=True)

Now that we have replaced all 0-values to NaN, let's check the overall median number of total employees.

In [ ]:
median_total_empl = df['total_empl'].median()
print(median_total_empl)

The following row replaces all NaN values in the Average Number of Employees column by the median value found above.

In [ ]:
df['total_empl'].fillna(median_total_empl, inplace=True)

Let's repeat this for the total payroll variable, but this time using a shorter syntax.

In [ ]:
df['total_wages'] = np.where(df['total_wages']==0, np.NaN, df['total_wages'])
df['total_wages'].fillna(df['total_wages'].median(), inplace=True)

Since we altered Total Employees and Total Earnings, let's redefine the "Average Earnings" feature.

In [ ]:
del df['avg_wage']
df['avg_wage'] = df['total_wages']/df['total_empl']

And let's check how the summary statistics have changed:

In [ ]:
df.describe(percentiles=[0.01,0.05,0.25,0.50,0.75,0.95,0.99])

For more examples of imputing missing values, see the [8_1_inference_imputing_missing_values](8_1_inference_imputing_missing_values.ipynb) notebook.

### Removing Outliers 

Some values of average monthly wage still seem impossible or very unlikely. Some outliers in particular have average wages far exceeding the 99th percentile or well below the 1st percentile. 

Usually, you would want to perform a "sanity check" on these values with someone who knows the data well. **It is never a good idea to drop observations without prior investigation!** Here, however, for the purposes of this exercise, we chose to drop these values.

Let's start by flagging outliers, check what share of the rows we would be deleting, and drop the outliers.

In [ ]:
# Find all rows where the average monthly wage is below 200 or above 50,000
outlier_rows = ((df['avg_wage'] < 200)|(df['avg_wage'] > 50000))
df[outlier_rows].head(10)

In [ ]:
# What share of rows are outliers?
nrows_wages = df.shape[0]
nrows_wages_outliers = df[outlier_rows].shape[0]
print('Share outlier rows: {}%'.format(float(nrows_wages_outliers)/nrows_wages))

In [ ]:
# Let's get rid of the outlier rows
df = df[~outlier_rows]

### Scaling of Values

Certain models will have issue with the distance between features such as number of employees and average wages. Number of employees is typically a number between 1 and 100 while average wages are usually between 1000 and 10,000. In order to circumvent this problem we can scale our features.

In [ ]:
# Example: let's scale average wages:
min_avg_wage = df['avg_wage'].min()
max_avg_wage = df['avg_wage'].max()
df['avg_wage_scl'] = (df['avg_wage']-min_avg_wage)/(max_avg_wage-min_avg_wage)

Let's compare the distributions of the scaled and unscaled features.

In [ ]:
df[['avg_wage', 'avg_wage_scl']].describe()

Let's do it again for Total Employees and Total Wages.

In [ ]:
min_val = df['total_empl'].min()
max_val = df['total_empl'].max()
df['total_empl_scl'] = (df['total_empl']-min_val)/(max_val-min_val)

min_val = df['total_wages'].min()
max_val = df['total_wages'].max()
df['total_wages_scl'] = (df['total_wages']-min_val)/(max_val-min_val)

The final dataframe now looks like this:

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

### Define Function

All above steps have been summarized into a Python function below. We also added a final step that writes the feature table to the database.

In the step-by-step approach above, all SQL queries were entirely hard coded. The Python function however uses parameters so you can easily reuse it. The function's parameters are:
- `year`: The year at which we are doing the prediction.
- `qtr`: The quarter at which we are doing the prediction.
- `delta_t`: The forward-looking window we chose when creating the model's labels. The default value is 1, which means we are prediction at a given time whether an employer will still exist one year later.
- `schema`: Your team schema, where the label table will be written. The default value is set to `myschema`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `db_name`: Database name. This is the name of the SQL database we are using. The default value is set to `db_name`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `hostname`: Host name. This is the host name for the SQL database we are using. The default value is set to `hostname`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `overwrite`: Whether you want the function to overwrite tables that already exist. Before writing a table, the function will check whether this table exists, and by default will not overwrite existing tables.

In [ ]:
def employer_wages_empl(year, qtr, delta_t=1, schema=myschema, 
                        db_name=db_name, hostname=hostname, overwrite=False):
    
    conn = psycopg2.connect(database=db_name, host = hostname) #database connection
    cursor = conn.cursor()
    
    # Let's check if the table already exists:
    cursor.execute('''
    SELECT * FROM information_schema.tables 
    WHERE table_name = 'features_wages_empl_{year}q{qtr}' 
    AND table_schema = '{schema}';
    '''.format(year=year, qtr=qtr, schema=schema))

    # Let's write table if it does not exist (or if overwrite = True)
    if not(cursor.rowcount) or overwrite:
        print("Creating table")
        
        # Create Temp Table
        sql = '''
        CREATE TEMP TABLE features_emprs AS
        SELECT CONCAT(ein, '-', seinunit, '-', empr_no) AS id, 
                empl_month1::int+empl_month2::int+empl_month3::int AS total_empl,
                total_wages
        FROM il_des_kcmo.il_qcew_employers
        WHERE year = {year} AND quarter = {qtr};
        COMMIT;
        '''.format(year=year, qtr=qtr, schema=schema)
        cursor.execute(sql)
        
        # Merge onto labels and load into Python
        sql = '''
        SELECT a.id, b.total_empl, b.total_wages
        FROM ada_18_uchi.labels_{year}q{qtr}_{year_pdelta}q{qtr} AS a
        LEFT JOIN features_emprs AS b
        ON a.id = b.id;
        '''.format(year=year, qtr=qtr, year_pdelta=year+delta_t, schema=schema)
        df = pd.read_sql(sql, conn)
        
        # Impute Missing Values
        df['total_empl'] = np.where(df['total_empl']<=0, np.NaN, df['total_empl'])
        df['total_empl'].fillna(df['total_empl'].median(), inplace=True)
        df['total_wages'] = np.where(df['total_wages']<=0, np.NaN, df['total_wages'])
        df['total_wages'].fillna(df['total_wages'].median(), inplace=True)
        
        # Create Feature Average Wage
        df['avg_wage'] = df['total_wages']/df['total_empl']
        
        # Remove Outliers
        outlier_rows = ((df['avg_wage'] < 200)|(df['avg_wage'] > 50000))
        df = df[~outlier_rows]
        
        # Scale Values
        min_val = df['total_empl'].min()
        max_val = df['total_empl'].max()
        df['total_empl_scl'] = (df['total_empl']-min_val)/(max_val-min_val)
        min_val = df['total_wages'].min()
        max_val = df['total_wages'].max()
        df['total_wages_scl'] = (df['total_wages']-min_val)/(max_val-min_val)
        min_val = df['avg_wage'].min()
        max_val = df['avg_wage'].max()
        df['avg_wage_scl'] = (df['avg_wage']-min_val)/(max_val-min_val)
        
        # Write Table to Database
        engine = create_engine('postgresql://{}/{}'.format(hostname, db_name))
        df.to_sql('features_wages_empl_{year}q{qtr}'.format(year=year, qtr=qtr), 
                  engine, schema=myschema, index=False, if_exists='replace', )
        sql = '''
        ALTER TABLE {schema}.features_wages_empl_{year}q{qtr} OWNER TO {schema}_admin;
        COMMIT;
        '''.format(year=year, qtr=qtr, schema=schema)
        cursor.execute(sql)
        
    else:
        print("Table already exists")
    
    cursor.close()
    
    sql = '''
    SELECT * FROM {schema}.features_wages_empl_{year}q{qtr} 
    '''.format(year=year, qtr=qtr, schema=schema)
    df = pd.read_sql(sql, conn)  
    
    return df

In [ ]:
df_wages_empl_2013q1 = employer_wages_empl(2013, 1)

In [ ]:
df_wages_empl_2013q1.describe(include='all')

In [ ]:
df_wages_empl_2014q1 = employer_wages_empl(2014, 1)

In [ ]:
df_wages_empl_2014q1.describe(include='all')

## Firm Wage Gap

One of the advantages of using microdata is that you have access to individual wages instead of only aggregate levels. Among the possible features that you can create using individual wages are metrics encapsulating the wage gap between the firm's highest and lowest earners. Here, we present two such metric: the range (difference between the min and max) and the interquartile range (difference between the 75th and 25th percentiles). 

### Step by Step Approach

In [ ]:
conn = psycopg2.connect(database=db_name, host = hostname)
cursor = conn.cursor()

Let's start by querying the Wage data at the year and quarter of interest.

In [ ]:
sql = '''
CREATE TEMP TABLE all_wages AS
SELECT CONCAT(ein, '-', seinunit, '-', empr_no) AS id, ssn, wage
FROM il_des_kcmo.il_wage
WHERE year = 2013 AND quarter = 1;
COMMIT;
'''
cursor.execute(sql)

How many observations does the resulting table have?

In [ ]:
pd.read_sql('select count(*) from all_wages;', conn)

Let's now merge the temp table we created onto the labels table, load that table into Python and visualize the first rows.

In [ ]:
sql = '''
CREATE TEMP TABLE labels_all_wages AS
SELECT a.id, b.ssn, b.wage
FROM ada_18_uchi.labels_2013q1_2014q1 AS a
LEFT JOIN all_wages AS b
ON a.id = b.id;
COMMIT;
'''
cursor.execute(sql)

In [ ]:
df = pd.read_sql('SELECT * FROM labels_all_wages LIMIT 10', conn)
df

We now have a table where the observation level is the individual employee. In order to get wage gap metrics at employer level, we are going to group by Employer ID and keep certain percentiles of the underlying wages. Let's write the SQL query and load that table into Python.

In [ ]:
sql = '''
CREATE TEMP TABLE wage_pcts AS
SELECT id,
        COUNT(DISTINCT ssn) AS nb_empl,
        MIN(wage) AS min_wage,
        PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY wage) AS wage_pct_25,
        PERCENTILE_CONT(0.50) WITHIN GROUP(ORDER BY wage) AS med_wage,
        PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY wage) AS wage_pct_75,        
        MAX(wage) AS max_wage
FROM labels_all_wages
GROUP BY id;
COMMIT;
'''
cursor.execute(sql)

In [ ]:
df = pd.read_sql('SELECT * FROM wage_pcts LIMIT 100', conn)

In [ ]:
df.head()

Now let's create the wage-gap metrics we need.

In [ ]:
sql = '''
SELECT *,
        max_wage - min_wage AS wage_range,
        wage_pct_75 - wage_pct_25 AS wage_intqtr_range
FROM wage_pcts;
'''
df = pd.read_sql(sql, conn)

In [ ]:
df.shape

In [ ]:
df.head(10)

Notice the several issues we run into:

XXXXXXXXXXX

Restricting to firms that do not run into the above issues (firms with at least 5 employees, for example) will reduce largely the scope of our analysis. Here, since we want to include these features in the model, we will make this restriction. But as noted before, always think twice before dropping a large number of observations. 

In [ ]:
sql = '''
SELECT *,
        max_wage - min_wage AS wage_range,
        wage_pct_75 - wage_pct_25 AS wage_intqtr_range
FROM wage_pcts
WHERE nb_empl >= 5;
'''
df = pd.read_sql(sql, conn)

In [ ]:
df.shape

In [ ]:
df.head()

### Define Function

All above steps have been summarized into a Python function below. We also added a final step that writes the feature table to the database.

In the step-by-step approach above, all SQL queries were entirely hard coded. The Python function however uses parameters so you can easily reuse it. The function's parameters are:
- `year`: The year at which we are doing the prediction.
- `qtr`: The quarter at which we are doing the prediction.
- `delta_t`: The forward-looking window we chose when creating the model's labels. The default value is 1, which means we are prediction at a given time whether an employer will still exist one year later.
- `min_size`: The minimum number of employees. As we saw above, wage gap characteristics make little sense for very small firms. The default value is set to 5.  
- `schema`: Your team schema, where the label table will be written. The default value is set to `myschema`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `db_name`: Database name. This is the name of the SQL database we are using. The default value is set to `db_name`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `hostname`: Host name. This is the host name for the SQL database we are using. The default value is set to `hostname`, defined in the [Python Setup](#Python-Setup) section of this notebook.
- `overwrite`: Whether you want the function to overwrite tables that already exist. Before writing a table, the function will check whether this table exists, and by default will not overwrite existing tables.

In [ ]:
def employer_wage_gap(year, qtr, min_size=5, delta_t=1,
                      schema=myschema, db_name=db_name, hostname=hostname, overwrite=False):
    
    conn = psycopg2.connect(database=db_name, host = hostname) #database connection
    cursor = conn.cursor()
    
    # Let's check if the table already exists:
    cursor.execute('''
    SELECT * FROM information_schema.tables 
    WHERE table_name = 'features_wage_gap_{year}q{qtr}_size{min_size}' 
    AND table_schema = '{schema}';
    '''.format(year=year, qtr=qtr, min_size=min_size, schema=schema))

    # Let's write table if it does not exist (or if overwrite = True)
    if not(cursor.rowcount) or overwrite:
        print("Creating table")
        
        # Create Temp Table
        sql = '''
        CREATE TEMP TABLE all_wages AS
        SELECT CONCAT(ein, '-', seinunit, '-', empr_no) AS id, ssn, wage
        FROM il_des_kcmo.il_wage
        WHERE year = {year} AND quarter = {qtr};
        COMMIT;

        CREATE TEMP TABLE labels_all_wages AS
        SELECT a.id, b.ssn, b.wage
        FROM ada_18_uchi.labels_{year}q{qtr}_{year_pdelta}q{qtr} AS a
        LEFT JOIN all_wages AS b
        ON a.id = b.id;
        COMMIT;     

        CREATE TEMP TABLE wage_pcts AS
        SELECT id,
                COUNT(DISTINCT ssn) AS nb_empl,
                MIN(wage) AS min_wage,
                PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY wage) AS wage_pct_25,
                PERCENTILE_CONT(0.50) WITHIN GROUP(ORDER BY wage) AS med_wage,
                PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY wage) AS wage_pct_75,        
                MAX(wage) AS max_wage
        FROM labels_all_wages
        GROUP BY id;
        COMMIT;
        
        DROP TABLE IF EXISTS {schema}.features_wage_gap_{year}q{qtr}_size{min_size};
        CREATE TABLE {schema}.features_wage_gap_{year}q{qtr}_size{min_size} AS
        SELECT *,
                max_wage - min_wage AS wage_range,
                wage_pct_75 - wage_pct_25 AS wage_intqtr_range
        FROM wage_pcts
        WHERE nb_empl >= {min_size};
        COMMIT;
        
        ALTER TABLE {schema}.features_wage_gap_{year}q{qtr}_size{min_size} OWNER TO {schema}_admin;
        COMMIT;
        '''.format(year=year, qtr=qtr, min_size=min_size, year_pdelta=year+delta_t, schema=schema)
        cursor.execute(sql)
        
    else:
        print("Table already exists")
    
    cursor.close()
    
    sql = '''
    SELECT * FROM {schema}.features_wage_gap_{year}q{qtr}_size{min_size}
    '''.format(year=year, qtr=qtr, min_size=min_size, schema=schema)
    df = pd.read_sql(sql, conn)  
    
    return df

In [ ]:
df_wage_gap_2013q1 = employer_wage_gap(2013, 1)

In [ ]:
df_wage_gap_2013q1.describe(include='all')

In [ ]:
df_wage_gap_2014q1 = employer_wage_gap(2014, 1)

In [ ]:
df_wage_gap_2014q1.describe(include='all')